In [1]:
import numpy as np
import pandas as pd
#读取数据
df_cate = pd.read_csv(r'C:\Users\luvy\Desktop\数分实战练习\Amazon\amazon_categories.csv')
df_pro = pd.read_csv(r'C:\Users\luvy\Desktop\数分实战练习\Amazon\amazon_products.csv')
# 依靠‘id’和‘category_id’连接两表
df = pd.merge(df_pro,df_cate,how='left',left_on='category_id',right_on='id')
# asin-商品编号；title-产品标题；imgurl-产品的图片地址；producturl-产品网址；satrs-评级；reviews-评论数；price-立即购买价格；
# list price-折扣前原价；categroy_id-类别名称；isBestseller-是否是畅销书；boughtinlastmonth-最近一个月的购买数据;categroy_name-种类名
print('='*50)
print('产品数据形状',df.shape)
print('产品的数据列',df.columns.tolist())
print('产品前五行数据\n',df.head(5))
key_word = {
    'asin':{
        '含义':'商品编号，可能会有重复或空值',
        '处理':'去重，确保唯一性'
    },
    'stars':{
        '含义':'评级，范围设定在1-5',
        '处理':'不在范围内的值用中位数填充'
    },
    'reviews':{
        '含义':'商品的评论数，不存在负数',
        '处理':'补0，极高值缩尾处理'
    },
    'price':{
        '含义':'折后价格，不存在<=0，极高值可能说明是爆款',
        '处理':'平均值，极高值缩尾处理'
    },
    'listPrice':{
        '含义':'折前价格，不存在负值，且对应商品折前应该比折后贵',
        '处理':'价格和price*1.1，极高值缩尾处理'
    },
    'boughtInLastMonth': {
        '含义': '月销量，可能存在负值，极端值',
        '处理': '替换负数为0，缩尾处理极端值'
    }
}

产品数据形状 (1426337, 13)
产品的数据列 ['asin', 'title', 'imgUrl', 'productURL', 'stars', 'reviews', 'price', 'listPrice', 'category_id', 'isBestSeller', 'boughtInLastMonth', 'id', 'category_name']
产品前五行数据
          asin                                              title  \
0  B014TMV5YE  Sion Softside Expandable Roller Luggage, Black...   
1  B07GDLCQXV  Luggage Sets Expandable PC+ABS Durable Suitcas...   
2  B07XSCCZYG  Platinum Elite Softside Expandable Checked Lug...   
3  B08MVFKGJM  Freeform Hardside Expandable with Double Spinn...   
4  B01DJLKZBA  Winfield 2 Hardside Expandable Luggage with Sp...   

                                              imgUrl  \
0  https://m.media-amazon.com/images/I/815dLQKYIY...   
1  https://m.media-amazon.com/images/I/81bQlm7vf6...   
2  https://m.media-amazon.com/images/I/71EA35zvJB...   
3  https://m.media-amazon.com/images/I/91k6NYLQyI...   
4  https://m.media-amazon.com/images/I/61NJoaZcP9...   

                             productURL  stars  reviews   

In [2]:
df = pd.DataFrame(df)

In [3]:
# 数据清洗
# 识别缺失值
print('='*10+'缺失值统计:'+'='*10+'\n',df.isnull().sum())
# 标题缺失填充'-'
df['title'] = df['title'].fillna('-')

==========缺失值统计:==========
 asin                 0
title                1
imgUrl               0
productURL           0
stars                0
reviews              0
price                0
listPrice            0
category_id          0
isBestSeller         0
boughtInLastMonth    0
id                   0
category_name        0
dtype: int64


In [4]:
# 异常值处理
print('='*15+'\n评价星级处理\n'+'='*15)
# 针对编号去重
len_df= len(df)
df = df.drop_duplicates(subset="asin", keep="first") 
print(f'去重前商品数量：{len_df},去重后商品数量：{len(df)}')

评价星级处理
去重前商品数量：1426337,去重后商品数量：1426337


In [5]:
# IQR异常值检测
def detect_outliers_IQR(data,column,q):
    Q1 = data[column].quantile(q)
    Q3 = data[column].quantile(1-q)
    IQR = Q3-Q1
    lower = Q1-1.5*IQR
    upper = Q3+1.5*IQR
    outliers = data[(data[column]<lower)&(data[column]>upper)]
    print('='*50)
    print(f'{column}的异常值检测情况：')
    print('='*50)
    print(f'Q1:{Q1:.2f},Q3:{Q3:.2f},IQR:{IQR:.2f}')
    print(f'正常值范围：({lower:.2f},{upper:.2f})')
    print(f'异常值数量：{len(outliers)}({len(outliers)/len(data)*100:.2f}%)')
    return outliers,lower,upper

In [6]:
# 针对评论
print('='*50+'\n评价星级处理\n'+'='*50)
df['stars'] = pd.to_numeric(df['stars'], errors='coerce')
vaild_stars = df[(df['stars']>=0)&(df['stars']<=5)]
df['stars'] = np.clip(df['stars'],0,5)
vaild_stars2 = df[(df['stars']>=0)&(df['stars']<=5)]
print(f'处理评级之前正常评级的商品数:{len(vaild_stars)}\n处理评级之后正常评级的商品数:{len(vaild_stars2)}')

评价星级处理
处理评级之前正常评级的商品数:1426337
处理评级之后正常评级的商品数:1426337


In [7]:
# 针对评论数
print('='*50+'\n评论数处理\n'+'='*50)
# 非负处理
vaild_reviews = df[df['reviews']<0]
df['reviews'] = np.where(df['reviews']<0,0,df['reviews'])
print(f'非负处理评论数的商品数:{len(vaild_reviews)}')
print(f'评论数统计：\n',df['reviews'].describe())
# 极端值处理（评论数极高属于右偏，先进行缩尾再进行对数处理，但可能存在爆款）
outliers_review,lower_review,upper_review = detect_outliers_IQR(df,'reviews',0.2)
if lower_review<0:
    lower_review=0
df['reviews'] = np.clip(df['reviews'],lower_review,upper_review)
print(f'评论数高于{upper_review},低于{lower_review}的商品数为:{len(outliers_review)}')
df['reviews_log'] = np.log1p(df['reviews'])
print(f'处理后数据数：{len(df)}条')

评论数处理
非负处理评论数的商品数:0
评论数统计：
 count    1.426337e+06
mean     1.807508e+02
std      1.761453e+03
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.465630e+05
Name: reviews, dtype: float64
reviews的异常值检测情况：
Q1:0.00,Q3:1.00,IQR:1.00
正常值范围：(-1.50,2.50)
异常值数量：0(0.00%)
评论数高于2.5,低于0的商品数为:0
处理后数据数：1426337条


In [8]:
# 针对折扣价格
df2 = df.copy()
print('='*50+'\n折扣价格（售价）处理\n'+'='*50)
# 非负处理删除记录
invaild_price_count = df[df['price']<0]
df = df[df['price']>=0]
zero_price = df[df['price']==0]
print(f'处理了{len(invaild_price_count)}条负价格,其中价格为0的商品有：{len(zero_price)}')
print(f'折扣价格统计：\n',df['price'].describe())
# 价格极端值缩尾处理
outliers_price,lower_price,upper_price = detect_outliers_IQR(df,'price',0.01)
df['price'] = np.clip(df['price'],lower_price,upper_price)
print(f'处理了{len(outliers_price)}条具有极端值价格商品')
print(f'处理后数据数：{len(df)}条')

折扣价格（售价）处理
处理了0条负价格,其中价格为0的商品有：32772
折扣价格统计：
 count    1.426337e+06
mean     4.337540e+01
std      1.302893e+02
min      0.000000e+00
25%      1.199000e+01
50%      1.995000e+01
75%      3.599000e+01
max      1.973181e+04
Name: price, dtype: float64
price的异常值检测情况：
Q1:0.00,Q3:400.99,IQR:400.99
正常值范围：(-601.49,1002.48)
异常值数量：0(0.00%)
处理了0条具有极端值价格商品
处理后数据数：1426337条


In [9]:
# 针对折前价格
print('='*50+'\n折前价格（标价）处理\n'+'='*50)
# 非负处理
invaild_listprice_count = df[df['listPrice']<0]
df = df[df['listPrice']>=0]
zero_listPrice = df[df['listPrice']==0]
print(f'处理了{len(invaild_listprice_count)}条负原价格，其中价格为0的商品数：{len(zero_listPrice)}')
print(f'折前价格统计：\n',df['listPrice'].describe())
# 极端值处理
outliers_Price,lower_Price,upper_Price = detect_outliers_IQR(df,'listPrice',0.01)
df['listPrice'] = np.clip(df['listPrice'],lower_Price,upper_Price)
print(f'处理了{len(outliers_Price)}条具有极端值价格商品')
print(f'处理后数据数：{len(df)}条')

折前价格（标价）处理
处理了0条负原价格，其中价格为0的商品数：1080503
折前价格统计：
 count    1.426337e+06
mean     1.244916e+01
std      4.611198e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      9.999900e+02
Name: listPrice, dtype: float64
listPrice的异常值检测情况：
Q1:0.00,Q3:199.95,IQR:199.95
正常值范围：(-299.92,499.87)
异常值数量：0(0.00%)
处理了0条具有极端值价格商品
处理后数据数：1426337条


In [10]:
print('='*50+'\n标价与售价关系处理\n'+'='*50)
Price = df[['price','listPrice']].copy()
# 向量化判断，price和listPrice之间的关系
mask1 = (Price['price']==0)&(Price['listPrice']==0)
mask2 = (Price['price']==0)&(Price['listPrice']>0)
mask3 = (Price['price']>0)&(Price['listPrice']==0)
mask4 = ~(mask1|mask2|mask3)
# 销售分类
Price.loc[mask1,'sale_category'] = '标价异常'
Price.loc[mask2,'sale_category'] = '商品下架'
Price.loc[mask3,'sale_category'] = '商品上新'
Price.loc[mask4,'sale_category'] = '正常销售'
# ‘标价异常’取nan，标价<售价，设标价为售价的1.1
Price.loc[mask4 & (Price['listPrice']<Price['price']),'listPrice'] = Price['price']*1.1
Price.loc[mask1,['price','listPrice']] = np.nan
Price[['price','listPrice']] = Price[['price','listPrice']].apply(pd.to_numeric,errors='coerce')
df[['price','listPrice','sale_category']] = Price
# 删除nan值所在行
abnormal_count = len(df[df['sale_category'] == '标价异常'])
df = df[df['sale_category'] != '标价异常']
print(f'删除了{abnormal_count}条标价异常商品数据')
print(f'处理后前五行数据：\n',df[['price','listPrice','sale_category']].head(5))

标价与售价关系处理
删除了32723条标价异常商品数据
处理后前五行数据：
     price  listPrice sale_category
0  139.99       0.00          商品上新
1  169.99     209.99          正常销售
2  365.49     429.99          正常销售
3  291.59     354.37          正常销售
4  174.99     309.99          正常销售


In [11]:
# 月销量
print('='*50+'\n月销量处理\n'+'='*50)
print(f'月销量统计：\n',df['boughtInLastMonth'].describe())
# 非负处理
invaild_sales = df[df['boughtInLastMonth']<0]
df['boughtInLastMonth'] = np.where(df['boughtInLastMonth']<0,0,df['boughtInLastMonth'])
print(f'处理了{len(invaild_sales)}条销售量不合理的商品')
# 极端值处理
outliers_sales,lower_sales,upper_sales = detect_outliers_IQR(df,'boughtInLastMonth',0.01)
if lower_sales<0:
    lower_sales = 0
df['boughtInLastMonth'] = np.clip(df['boughtInLastMonth'],lower_Price,upper_Price)
print(f'处理了{len(outliers_sales)}条具有极端值价格商品')
df['boughtInLastMonth'] = np.log1p(df['boughtInLastMonth'])
print(f'处理后数据数：{len(df)}条')

月销量处理
月销量统计：
 count    1.393614e+06
mean     1.437831e+02
std      8.424685e+02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      5.000000e+01
max      1.000000e+05
Name: boughtInLastMonth, dtype: float64
处理了0条销售量不合理的商品
boughtInLastMonth的异常值检测情况：
Q1:0.00,Q3:2000.00,IQR:2000.00
正常值范围：(-3000.00,5000.00)
异常值数量：0(0.00%)
处理了0条具有极端值价格商品
处理后数据数：1393614条


In [12]:
# 衍生数据特征
# 评级（0，2，3，4，4.5，5）
df['stars_categray'] = pd.cut(
    df['stars'],
    bins = [0,2,3,4,4.5,5],
    labels = ['很差','合格','良好','很好','优秀'],
    include_lowest = True
)
# 折扣
df['price_rate'] = np.where(
    df['listPrice'],
    np.round((df['listPrice']-df['price'])/df['listPrice'],2),
    0
)
# 价格分段
df['price_categray'] = pd.cut(
    df['price'],
    bins = [0,100,250,500,800,1500],
    labels = ['低价','中低价','中价','中高价','高价'],
    include_lowest = True
)
df['listPrice_categray'] = pd.cut(
    df['listPrice'],
    bins = [0,100,250,500,800,1500],
    labels = ['低价','中低价','中价','中高价','高价'],
    include_lowest = True
)
# 产品热度指标
df['popularity_score'] = (df['reviews'] + df['boughtInLastMonth']) / 2

In [13]:
# 数据质量验证
print('='*50+'\n数据质量验证\n'+'='*50)
print(f'记录了{len(df)}条数据')
print(f'特征数量：{len(df.columns)}')
# 非负值检查
print('-'*10+'\n非负值检查\n'+'-'*10)
number_check = {
    'stars':(0,5),
    'reviews':(0,np.inf),
    'price':(0,np.inf),
    'lastPrice':(0,np.inf),
    'boughtInLastMonth':(0,np.inf)
}
for col,(min,max) in number_check.items():
    if col in df:
        number_false = len(df[(df[col]<min)|(df[col]>max)])
    print(f'{col}特征存在{number_false}条异常值')
print('-'*10+'\n逻辑一致性检查\n'+'-'*10)
price_issues = len(df[(df['price']>df['listPrice'])&(df['sale_category']=='正常销售')])
discount_issues = len(df[(df['price_rate']<0)|(df['price_rate']>1)])
print(f'存在{price_issues}件商品的标价与售价之间异常关系')
print(f'存在{discount_issues}件商品折扣问题')
print('-'*10+'\n缺失值检查\n'+'-'*10)
print(df.isnull().sum())

数据质量验证
记录了1393614条数据
特征数量：20
----------
非负值检查
----------
stars特征存在0条异常值
reviews特征存在0条异常值
price特征存在0条异常值
lastPrice特征存在0条异常值
boughtInLastMonth特征存在0条异常值
----------
逻辑一致性检查
----------
存在0件商品的标价与售价之间异常关系
存在0件商品折扣问题
----------
缺失值检查
----------
asin                  0
title                 0
imgUrl                0
productURL            0
stars                 0
reviews               0
price                 0
listPrice             0
category_id           0
isBestSeller          0
boughtInLastMonth     0
id                    0
category_name         0
reviews_log           0
sale_category         0
stars_categray        0
price_rate            0
price_categray        0
listPrice_categray    0
popularity_score      0
dtype: int64


In [14]:
# 数据可视
import matplotlib
matplotlib.use('TKAgg')
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

df1 = df.copy()
plt.figure(figsize=(16,15))
plt.subplot(2,2,1)
# stars-饼图
sum_asin = df1['asin'].count()
Stars = df1.groupby('stars_categray',observed=False).agg(
    商品数=('asin','count'),
    占比=('asin',lambda x:round(x.count()*100/sum_asin,2))
)
Stars
plt.pie(Stars['占比'],
        labels=Stars.index,
         colors=['#F7A6AC','#F7B2C7','#F3BBB1','#EEC78A','#EEE9A2'],
         autopct='%1.1f%%',
         shadow=True
        )
plt.title('Amazon商品评价星级占比')
plt.legend(loc='upper left',fontsize=7)

# 价格分布
plt.subplot(2,2,2)
price = df1.groupby('price_categray', observed=False)['asin'].count()
price

bars = plt.bar(
    price.index,
    price.values,
    width=0.8,
    align='center',
    color='#EEC78A',
    label='售价',
)
for bar in bars:
    height = bar.get_height() 
    plt.text(
        x=bar.get_x() + bar.get_width() / 2,
        y=height + 1,
        s=f'{height}',
        ha='center',
        va='bottom',
        fontsize=8
    )
plt.xlabel('价格区间')
plt.ylabel('商品数')
plt.title('Amazon商品售价')
plt.legend()
plt.grid('x')

# 销量分布
plt.subplot(2,2,3)
bought_categray = df.groupby('category_name')['boughtInLastMonth'].count()
plt.bar(bought_categray.index,bought_categray.values,alpha=0.7, color='lightgreen')

plt.title('月销量分布')
plt.xticks([])
plt.xlabel('商品分类')
plt.ylabel('总销量（log）')
reslut = df1.sort_values('boughtInLastMonth').head(3)
plt.figtext(
    x=0.011,y=0.01,
    s=f'注：商品分类总销售量前三的类别：{list(bought_categray.head(3).index)}\n商品销售量前三的商品编号是：{list(reslut.asin)}',
    fontsize=10,
    color='black',
    ha='left'
)

# 畅销品占比
plt.subplot(2,2,4)
Is_BestSeller = df1['isBestSeller'].value_counts()
Is_BestSeller.index = ['畅销品' if index else '非畅销品' for index in Is_BestSeller.index]
plt.pie(
        Is_BestSeller.values,
        labels=Is_BestSeller.index,
         colors=['#F7A6AC','#EEE9A2'],
         autopct='%1.1f%%',
         shadow=True
)
plt.title('畅销品占比（%）')
plt.legend(loc='upper right',fontsize=7)
plt.suptitle('Amazon商品分析可视化', fontsize=15, fontweight='bold', y=0.99)
plt.tight_layout(pad=3.0, h_pad=3.0, w_pad=2.0)  # 增加子图间距
plt.subplots_adjust(top=0.92) 
plt.show()

# 打印关键统计信息
print("=" * 50)
print("关键业务指标统计")
print("=" * 50)
print(f"总商品数量: {len(df1):,}")
print(f"畅销品比例: {Is_BestSeller['畅销品']/len(df1)*100:.2f}%")
print(f"平均评分: {df1['stars'].mean():.2f}")
print(f"平均价格: ${df1['price'].mean():.2f}")
print(f"总月销量: {df1['boughtInLastMonth'].sum():,}")

关键业务指标统计
总商品数量: 1,393,614
畅销品比例: 0.61%
平均评分: 4.01
平均价格: $42.23
总月销量: 2,484,088.4155837977


In [84]:
# # 价格分布
# plt.subplot(2,2,2)
# price = df1.groupby('price_categray', observed=False)['asin'].count()
# price
# plt.figure(figsize=(8,6))

# bars = plt.bar(
#     price.index,
#     price.values,
#     width=0.8,
#     align='center',
#     color='#EEC78A',
#     label='售价',
# )
# for bar in bars:
#     height = bar.get_height() 
#     plt.text(
#         x=bar.get_x() + bar.get_width() / 2,
#         y=height + 1,
#         s=f'{height}',
#         ha='center',
#         va='bottom',
#         fontsize=8
#     )
# plt.xlabel('价格区间')
# plt.ylabel('商品数')
# plt.title('Amazon商品售价')
# plt.legend()
# plt.grid('x')


In [85]:
# # 销量分布
# plt.subplot(2,2,3)
# bought_categray = df.groupby('category_name')['boughtInLastMonth'].count()
# plt.figure(figsize=(10,6))
# plt.bar(bought_categray.index,bought_categray.values,alpha=0.7, color='lightgreen')

# plt.title('月销量分布')
# plt.xticks([])
# plt.xlabel('商品分类')
# plt.ylabel('总销量（log）')
# reslut = df1.sort_values('boughtInLastMonth').head(3)
# plt.figtext(
#     x=0.1,y=0.01,
#     s=f'注：商品分类总销售量前三的类别：{list(bought_categray.head(3).index)}\n商品销售量前三的商品编号是：{list(reslut.asin)}',
#     fontsize=10,
#     color='black',
#     ha='left'
# )



Text(0.1, 0.01, "注：商品分类总销售量前三的类别：['Abrasive & Finishing Products', 'Accessories & Supplies', 'Additive Manufacturing Products']\n商品销售量前三的商品编号是：['B097Y1P9QY', 'B0714BLGCR', 'B0BR9T2LKG']")

In [87]:
# # 畅销品占比
# plt.subplot(2,2,4)
# Is_BestSeller = df1['isBestSeller'].value_counts()
# Is_BestSeller.index = ['畅销品' if index else '非畅销品' for index in Is_BestSeller.index]
# plt.pie(
#         Is_BestSeller.values,
#         labels=Is_BestSeller.index,
#          colors=['#F7A6AC','#EEE9A2'],
#          autopct='%1.1f%%',
#          shadow=True
# )
# plt.title('畅销品占比（%）')
# plt.legend()
# plt.suptitle('Amazon商品分析可视化')
# plt.show()

In [32]:
# 价格与销量
relation = df.groupby('category_name').agg(
    mean_price=('price','mean'),
    stars_mean=('stars','mean')
)
plt.scatter(relation.index,relation['mean_price'],marker='o',color='#F7A6AC')
plt.title('Amazon商品种价格与销量之间的关系')
plt.xlabel('价格')
plt.ylabel('销量')
plt.xticks([])
plt.show()
# 评分与销量
plt.scatter(relation.index,relation['stars_mean'],marker='o',color='#F7A6AC')
plt.title('Amazon商品种评分与销量之间的关系')
plt.xlabel('评分')
plt.ylabel('销量')
plt.xticks([])
plt.show()

In [107]:
df.to_csv(r'C:\Users\luvy\Desktop\数分实战练习\Amazon\amazon.csv',index=False)
print('数据已成功保存！')

数据已成功保存！
